<a href="https://colab.research.google.com/github/sanjaynagi/AnoFold/blob/main/notebooks/AnoFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %pip install py3Dmol

# !wget https://github.com/gnina/gnina/releases/download/v1.1/gnina
# !chmod +x gnina
# !./gnina --version

In [2]:
# # genes = ['AGAP006227', 'AGAP006228', 'AGAP006723', 'AGAP006724', 'AGAP006725', 'AGAP006726', 'AGAP006727']
# df_genes = pd.read_csv("https://raw.githubusercontent.com/sanjaynagi/AnoExpress/main/resources/AgamP4.annots.tsv", 
#                        sep="\t")

# search = 'S-transferase'
# genes = df_genes.fillna('bl').query('GeneDescription.str.contains(@search)')['GeneID'].to_list()

In [3]:
# ligands = ['ddt3d', 'deltamethrin3d']           #, 'pirimiphos-methyl-oxon3d', 'malathion3d', 'cis-permethrin3d','transfluthrin3d']
# genes = ['AGAP006227']
# # for gene in genes:
# #     for ligand in ligands:
# #           res = invin(gene, ligand=ligand, override_desc='glutathione transferase')

## iMOVE - inverse molecular docking with alphafold and gnina

In [4]:
import imove

mutagen_dicts = [None, {114:'T'}, {119:'V'}, {120:'L'}]

In [5]:
# for ligand in ['ddt3d', 'deltamethrin3d']:
#     for mutagen_dict in mutagen_dicts:
        
#         # if I114T, the motif needs to be adjusted to include the new mutation
#         override_motif = None
#         if mutagen_dict:
#             if list(mutagen_dict.keys())[0] == 114:
#                 override_motif = ('RFTFERI', "O", 4)

#         imove.dock("AGAP009194", ligand, wkdir="../../", verbose=True, mutagenesis_dict=mutagen_dict, override_motif=override_motif)

In [2]:
# docked = imove.dock("AGAP000818", "ddt3d", wkdir="../../", verbose=True, mutagenesis_dict=mutagen_dicts[3], override_motif=None)

In [1]:
import imove

imove.download_pdb('AGAP000818', wkdir="../../", verbose=True, p450=True)

Receptor file not found. Downloading and preparing AlphaFold PDB for AGAP000818
Successfully downloaded 1TQN.pdb to ./1TQN.pdb
INFO:
available bundle cache has not been initialized yet
INFO:
UCSF ChimeraX version: 1.8 (2024-06-10)
INFO:
© 2016-2024 Regents of the University of California.  All rights reserved.
INFO:
Executing: open temp_chimerax_script.cxc
INFO:
Executing: open ../../receptors/AGAP000818_raw.pdb
INFO:
_AGAP000818_raw.pdb_ title:  
**Alphafold monomer V2.0 prediction for cytochrome P450 CYP9K1 (Q6VFG9)**
[[more info...]](cxcmd:log metadata #1)  
  

INFO:
Chain information for AGAP000818_raw.pdb #1  
---  
Chain | Description | UniProt  
[A](cxcmd:select /A:1-192 "Select chain") | [cytochrome P450 CYP9K1](cxcmd:sequence chain #1/A "Show sequence") | [Q6VFG9_ANOGA](cxcmd:open Q6VFG9 from uniprot associate #1/A "Show annotations") [1-192](cxcmd:select #1/A:1-192 "Select sequence")  
  

INFO:
Executing: open ./1TQN.pdb
INFO:
_1TQN.pdb_ title:  
**Crystal structure of huma

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ../../receptors/AGAP000818_raw.pdb)



In [2]:
imove.vectorbase_to_uniprot("AGAP000818")

'Q6VFG9'

In [69]:
import requests
def vectorbase_to_uniprot(gene_id):
    url = "https://rest.uniprot.org/idmapping/run"
    data = {
        "from": "VEuPathDB",
        "to": "UniProtKB",
        "ids": f"VectorBase:{gene_id}"
    }

    response = requests.post(url, data=data)
    response.raise_for_status()
    job_id = response.json()["jobId"]

    status_url = f"https://rest.uniprot.org/idmapping/status/{job_id}"
    while True:
        status_response = requests.get(status_url)
        status_response.raise_for_status()
        status = status_response.json()
        if "jobStatus" in status and status["jobStatus"] in ("RUNNING", "NEW"):
            continue
        elif "results" in status or "failedIds" in status:
            break

    results_url = f"https://rest.uniprot.org/idmapping/stream/{job_id}"
    results_response = requests.get(results_url)
    results_response.raise_for_status()
    results = results_response.json()['results']

    # if multiple transcripts, choose the longest
    lens = {}
    if len(results) > 1:
        for entry in results:
            gene_data = get_uniprot_data(entry['to'], vectorbase=False)
            lens[entry['to']] = gene_data['length']

        max_key = max(lens, key=lens.get)
        print(f"selecting longest transcript for gene_id: {gene_id}, {max_key} : {lens[max_key]}")
        return max_key
    else:
        return results[0]["to"]

def get_uniprot_data(gene_id, vectorbase=True):
    if vectorbase:
        uniprot_acc = vectorbase_to_uniprot(gene_id)
    else:
        uniprot_acc = gene_id
    if not uniprot_acc:
        return f"No UniProt accession found for VectorBase ID: {gene_id}"

    # UniProt API endpoint
    base_url = "https://rest.uniprot.org/uniprotkb/search"

    print(f"UniProt accession: {uniprot_acc}")

    # Query parameters
    params = {
        "query": f"accession:{uniprot_acc}",
        "format": "json",
        "fields": "gene_names,protein_name,organism_name,go, sequence"
    }

    # Send request with retry mechanism
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            if data['results']:
                result = data['results'][0]
                return {
                    "gene_name": result.get('genes', [{}])[0].get('geneName', {}).get('value', 'N/A'),
                    "protein_name": result.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A'),
                    "organism": result.get('organism', {}).get('scientificName', 'N/A'),
                    "function": next((comment['texts'][0]['value'] for comment in result.get('comments', []) if comment['commentType'] == 'FUNCTION'), 'N/A'),
                    "go_terms": [go['id'] for go in result.get('goTerms', [])],
                    "length": result.get('sequence', {}).get('length', 'N/A')  # Added this line to get the length
                }
            else:
                return f"No data found for UniProt accession: {uniprot_acc}"

        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                return f"Error retrieving data: {str(e)}"
            time.sleep(2 ** attempt)  # Exponential backoff

    return "Max retries reached. Unable to retrieve data."


In [70]:
results = vectorbase_to_uniprot("AGAP000818")

UniProt accession: Q6VFG9
UniProt accession: Q6VFH1
UniProt accession: Q6VFH2
UniProt accession: Q6VFH3
UniProt accession: Q8T633
selecting longest transcript for gene_id: AGAP000818, Q8T633 : 531


In [31]:
get_uniprot_data('Q8T633', vectorbase=False)

UniProt accession: Q8T633


{'gene_name': 'cyp9k1',
 'protein_name': 'N/A',
 'organism': 'Anopheles gambiae',
 'function': 'N/A',
 'go_terms': [],
 'length': 531}